In [1]:
import random
import pandas as pd
import numpy as np
import pickle
import cvxpy as cp
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
from tqdm import tqdm
import os
from concurrent.futures import ThreadPoolExecutor


"""pathの定義"""
DIR_HOURLY = 'Z:/lab/lifebehavior/2020_jikoku_danjonenso.csv'
DIR_FULL_DAY = 'Z:/lab/lifebehavior/2020_4shihyo_danjonenso.csv'
DIR_GENRE = 'Z:/lab/lifebehavior/nhk_genre.csv'



def calc_list_diff(list1:list, list2:list):
    # リストの長さが異なる場合のエラーチェック
    if len(list1) != len(list2):
        raise ValueError("リストの長さが異なります。")
    
    # 各要素の差分を計算して新しいリストを生成
    difference = [a - b for a, b in zip(list1, list2)]
    return difference

# not P_iの算出による P_iの算出
def get_P_i(hourly_table:pd.DataFrame,Transition_rates:list,activity_ordering:list):
    t_0_table = sort_df_by_list(hourly_table[(hourly_table["Time"].dt.hour == 0) & (hourly_table["Time"].dt.minute == 0)],"Activity",activity_ordering).copy()
    initial_rate = np.array(t_0_table["Rate"])

    # 正規化
    initial_rate = initial_rate / np.sum(initial_rate)

    P_i_list = []
    for i in range(len(activity_ordering)):

        # 行列の加工
        n = len(initial_rate)
        one_vector = np.ones(n)
        modified_transition_rates = []

        for A in Transition_rates:
            A_i = A.copy()
            A_i[i, :] = 0  # 行 i を全て 0 に置き換える
            modified_transition_rates.append(A_i)

        # 初期行為者率ベクトルの加工
        modified_initial_rate = initial_rate.copy()
        modified_initial_rate[i] = 0

        # 最終的な行為者率ベクトルの計算
        final_rate = modified_initial_rate

        for A_i in modified_transition_rates:
            final_rate = A_i @ final_rate

        # 行為 i を一度も行わない確率の計算
        P_not_i = one_vector @ final_rate

        P_i = 1-P_not_i

        # 独自定義のラベルを適用しているため、重複集計の関係上どうしても例外が発生する。
        if P_i < 0:
            P_i = 0

        P_i_list.append(P_i)

    return P_i_list

# データフレームを参照リストに基づいてソートするヘルパー関数
def sort_df_by_list(df, column, reference_list):
    return df.set_index(column).reindex(reference_list).reset_index()

# 遷移率を計算する関数
def calc_transition_rates(beta_i: np.ndarray, time_list: list, time_table: pd.DataFrame, activity_list: list):
    transition_rates = []
    middle = activity_list  # 活動のリストを基準として使用
    
    for i in range(len(time_list)):
        time_t = time_list[i]
        time_t1 = time_list[(i+1) % len(time_list)]  # 次の時間帯
        
        # 両方の時間帯の行動率を取得
        y_t = np.array(sort_df_by_list(time_table[(time_table["Time"].dt.hour == time_t.hour) & (time_table["Time"].dt.minute == time_t.minute) ], "Activity", middle)["Rate"])
        y_t1 = np.array(sort_df_by_list(time_table[(time_table["Time"].dt.hour == time_t1.hour) & (time_table["Time"].dt.minute == time_t1.minute) ], "Activity", middle)["Rate"])
        
        # 変数の定義
        no_of_elem = len(middle)
        a = cp.Variable((no_of_elem, no_of_elem), nonneg=True)  # 非負の要素を持つ遷移行列

        # 目的関数と制約条件の定義
        delta = np.eye(no_of_elem)  # クロネッカーのデルタ
        # 目的関数: (1 - beta_i * delta) で修正された、二乗差の合計を最小化
        objective = cp.Minimize(cp.sum(cp.multiply(1 - beta_i * delta, cp.square(a))))

        # 行の和が1になる制約に変更
        constraints = [
            cp.sum(a, axis=1) == 1,  # 遷移行列の各行の和が1になる制約
            a >= 0,  # 非負制約
        ]

        # 行動率を正規化
        y_t = y_t / np.sum(y_t)
        y_t1 = y_t1 / np.sum(y_t1)

        # 次の時間帯の行動率と遷移行列の適用が一致する制約
        constraints.append(y_t1 - a.T @ y_t == 0)

        # 問題を定義する
        problem = cp.Problem(objective, constraints)

        # ソルバーを使用して問題を解く（別のソルバーを指定して解く）
        problem.solve(solver=cp.SCS, verbose=False)

        # 得られた遷移行列を格納
        transition_rates.append(a.value)

    return np.array(transition_rates)

def roulette_wheel_selection(weights:list,labels:list):
    """
    Performs roulette wheel selection on a list of weights.
    
    Args:
    weights (list of float): The weights or probabilities for each item.
    
    Returns:
    int: The index of the selected item.
    """
    if (len(weights) != len(labels)):
        print("weights and labels has different length @roulette_wheel_selection")
        os._exit(1)
    # Calculate the cumulative sum of weights
    cumulative_sum = [sum(weights[:i+1]) for i in range(len(weights))]
    total_sum = cumulative_sum[-1]
    
    # Generate a random number in the range [0, total_sum)
    random_num = random.uniform(0, total_sum)
    
    # Find the index where the random number would fit in the cumulative sum
    for i, cum_sum in enumerate(cumulative_sum):
        if random_num < cum_sum:
            return i, labels[i]
        
# 目的関数
def objective(beta_i:np.ndarray, time_list:list, time_table:pd.DataFrame, activity_list:list,orig_P_i:list):
    transition_rates = calc_transition_rates(beta_i, time_list, time_table, activity_list)
    calclated_P_i = get_P_i(time_table,transition_rates,activity_list) # 一日のユニークな行為者率の算出
    diff_1 = [e for e in calc_list_diff(orig_P_i,calclated_P_i)]
    diff_2 = sum([e**2 for e in diff_1])
    return diff_2,diff_1

# 各粒子の評価を並列で行う関数
def evaluate_particle(objective, particle_position, time_list, time_table, activity_ordering, orig_P_i):
    return objective(particle_position, time_list, time_table, activity_ordering, orig_P_i)[0]

# βのパラメータフィッティング
def pso(objective, bounds, n_iterations, num_particles, w, c1, c2, time_list, time_table, activity_ordering, orig_P_i):
    # 粒子の位置と速度を初期化
    num_dimensions = len(activity_ordering)
    particles_position = np.random.uniform(bounds[:, 0], bounds[:, 1], (num_particles, num_dimensions))
    particles_velocity = np.random.uniform(-1, 1, (num_particles, num_dimensions))

    # 粒子の最良位置と全体の最良位置を初期化
    personal_best_position = particles_position.copy()
    
    # 並列で初期評価を計算
    with ThreadPoolExecutor() as executor:
        personal_best_eval = list(executor.map(lambda p: evaluate_particle(objective, p, time_list, time_table, activity_ordering, orig_P_i), particles_position))
    
    global_best_position = personal_best_position[np.argmin(personal_best_eval)]
    global_best_eval = np.min(personal_best_eval)

    print(f"Starting PSO >>> Initial Global Best Evaluation : {global_best_eval}")

    for i in tqdm(range(n_iterations)):
        for j in range(num_particles):
            # 粒子の速度を更新
            r1 = np.random.rand(num_dimensions)
            r2 = np.random.rand(num_dimensions)
            particles_velocity[j] = (w * particles_velocity[j] +
                                     c1 * r1 * (personal_best_position[j] - particles_position[j]) +
                                     c2 * r2 * (global_best_position - particles_position[j]))
            
            # 粒子の位置を更新
            particles_position[j] += particles_velocity[j]
            particles_position[j] = np.clip(particles_position[j], bounds[:, 0], bounds[:, 1])  # 境界のチェック
        
        # 並列で各粒子の評価を計算
        with ThreadPoolExecutor() as executor:
            current_evals = list(executor.map(lambda p: evaluate_particle(objective, p, time_list, time_table, activity_ordering, orig_P_i), particles_position))
        
        # 粒子の最良位置を更新
        for j in range(num_particles):
            if current_evals[j] < personal_best_eval[j]:
                personal_best_position[j] = particles_position[j].copy()
                personal_best_eval[j] = current_evals[j]
            
            # 全体の最良位置を更新
            if current_evals[j] < global_best_eval:
                global_best_position = particles_position[j].copy()
                global_best_eval = current_evals[j]
                print(f"Global Best Evaluation is Updated [Iteration {i+1}] : {global_best_eval}")

    return global_best_position, global_best_eval

(CVXPY) Sep 18 09:36:48 PM: Encountered unexpected exception importing solver OSQP:
ImportError('DLL load failed while importing qdldl: 指定されたモジュールが見つかりません。')


In [2]:
# 独自定義のラベルを適用するためにまず辞書を作る
small = [
    "睡眠", "食事", "身のまわりの用事", "療養・静養", "仕事", "仕事のつきあい", 
    "授業・学内の活動", "学校外の学習", "炊事・掃除・洗濯", "買い物", 
    "子どもの世話", "家庭雑事", "通勤", "通学", "社会参加", "会話・交際", 
    "スポーツ", "行楽・散策", "趣味・娯楽・教養(インターネット除く)", 
    "趣味・娯楽・教養のインターネット(動画除く)", "インターネット動画", 
    "テレビ", "録画番組・DVD", "ラジオ", "新聞", "雑誌・マンガ・本", 
    "音楽", "休息", "その他", "不明"
]
label_master = pd.read_csv(DIR_GENRE)
label_dict = {}
for k,v in zip(label_master["小分類"],label_master["モデル用定義"]):
    label_dict[k]=v

# 行動種別の並び順管理
ACTIVITY_ORDERING = ['睡眠', '食事', '身のまわりの用事', '療養・静養', '仕事', '仕事のつきあい', '授業・学内の活動',
       '学校外の学習', '炊事・掃除・洗濯', '買い物', '子どもの世話', '家庭雑事', '通勤', '通学', '社会参加',
       '会話・交際', 'スポーツ', '行楽・散策', '趣味・娯楽・教養', 'マスメディア接触', '休息']

# label_master["モデル用定義"].unique()


# PSOのパラメータ
num_particles = 50  # 粒子の数
n_iterations = 10  # 反復回数
w = 0.5  # 慣性項
c1 = 1.5  # 粒子の最良位置への係数
c2 = 1.5  # 全体の最良位置への係数

for label in tqdm([
        '男１０代', '男２０代', '男３０代', '男４０代', '男５０代', '男６０代', '男７０歳以上',
        '女１０代', '女２０代', '女３０代', '女４０代', '女５０代', '女６０代', '女７０歳以上']):
    for day in (["平日","土曜日","日曜日"]):

        # 時間帯ごとの行為者率テーブルの処理
        data_hourly = pd.read_csv(DIR_HOURLY)
        data_hourly = data_hourly.set_axis(["Day","Group","Activity","Time","Rate"],axis=1)
        data_hourly["Time"] = pd.to_datetime(data_hourly["Time"], format="%H:%M")
        data_hourly = data_hourly.query("Group == @label & Activity in @small & Day == @day")
        data_hourly["Activity"] = data_hourly["Activity"].map(label_dict) # 独自定義のラベル適用
        data_hourly_cleaned = data_hourly[["Activity","Time","Rate"]].groupby(["Time","Activity"],as_index=False).sum().copy()
        data_hourly_cleaned = data_hourly_cleaned.sort_values(by="Time")
        # 一日の行為者率のテーブルの処理
        data_full_day = pd.read_csv(DIR_FULL_DAY)
        df1 = data_full_day.copy()
        data_full_day = data_full_day.set_axis(["Day","Group","Activity","DailyRate","NetAverageTime","GrossAverageTime","Gross_SD"],axis=1)
        df2 = data_full_day.copy()
        data_full_day = data_full_day.query("Group == @label & Activity in @small & Day == @day")
        df3 = data_full_day.copy()
        data_full_day["Activity"] = data_full_day["Activity"].map(label_dict) # 独自定義のラベル適用
        data_full_day_cleaned = data_full_day[["Activity","DailyRate"]].groupby(["Activity"],as_index=False).sum().copy()
        df4 = data_full_day.copy()
        data_full_day_cleaned = sort_df_by_list(data_full_day_cleaned,"Activity",ACTIVITY_ORDERING)
        Orig_P_i = (data_full_day_cleaned["DailyRate"]/100).to_list()
        # アクティビティと時間のリストを取得
        time_list_hourly = data_hourly_cleaned['Time'].unique()


        # パラメータの設定
        bounds = np.array([[0.0, 1.0] for _ in range(len(ACTIVITY_ORDERING))])

        # 最適化の実行
        best_solution, best_evaluation = pso(objective=objective, bounds=bounds, n_iterations=n_iterations, 
                                            num_particles=num_particles, w=w, c1=c1, c2=c2, 
                                            time_list=time_list_hourly, time_table=data_hourly_cleaned, 
                                            activity_ordering=ACTIVITY_ORDERING, orig_P_i=Orig_P_i)
        
        # pickleで保存
        with open(f"Z:/lab/lifebehavior/βパラメータ_{label}_{day}-{str(best_evaluation)}-.pickle", 'wb') as f:
            pickle.dump(best_solution, f)
        transition_rates = calc_transition_rates(best_solution, time_list_hourly, data_hourly_cleaned, ACTIVITY_ORDERING)
        with open(f"Z:/lab/lifebehavior/遷移確率_{label}_{day}-{str(best_evaluation)}-.pickle", 'wb') as f:
            pickle.dump(transition_rates, f)

  0%|          | 0/14 [00:00<?, ?it/s]

Starting PSO >>> Initial Global Best Evaluation : 0.3884906303148565


Global Best Evaluation is Updated [Iteration 1] : 0.37652479437014214
Global Best Evaluation is Updated [Iteration 1] : 0.36792283736571124
Global Best Evaluation is Updated [Iteration 1] : 0.35598947811020193
Global Best Evaluation is Updated [Iteration 1] : 0.3263701804174426
Global Best Evaluation is Updated [Iteration 1] : 0.3070118432752413


Global Best Evaluation is Updated [Iteration 2] : 0.28148442900915815
Global Best Evaluation is Updated [Iteration 2] : 0.16680647686634303
Global Best Evaluation is Updated [Iteration 2] : 0.1242840704067568


Global Best Evaluation is Updated [Iteration 3] : 0.10404409821966126
Global Best Evaluation is Updated [Iteration 3] : 0.1020916484359986
Global Best Evaluation is Updated [Iteration 3] : 0.09555940748608992


...

Global Best Evaluation is Updated [Iteration 9] : 0.3244791479445973


  7%|▋         | 1/14 [1:03:04<13:40:04, 3784.93s/it]

Starting PSO >>> Initial Global Best Evaluation : 0.7191249931161628


Global Best Evaluation is Updated [Iteration 1] : 0.7104623418370845
Global Best Evaluation is Updated [Iteration 1] : 0.6748987318727592
Global Best Evaluation is Updated [Iteration 1] : 0.6626716167945698


Global Best Evaluation is Updated [Iteration 2] : 0.6607499298382751
Global Best Evaluation is Updated [Iteration 2] : 0.657215346843729
Global Best Evaluation is Updated [Iteration 2] : 0.606219002484491
Global Best Evaluation is Updated [Iteration 2] : 0.6061967832438108
Global Best Evaluation is Updated [Iteration 2] : 0.6008017298083803
Global Best Evaluation is Updated [Iteration 2] : 0.6002553397051358
Global Best Evaluation is Updated [Iteration 2] : 0.5634110687553702
